In [1]:
import itertools
from tqdm import tqdm
import collections

In [ ]:
n_blowups = 7

N = ToricLattice(n_blowups + 1)
K = N([-3] + [1]*n_blowups) # canonical divisor
E = identity_matrix(QQ, n_blowups+1)[:,1:].columns() # E is the set of exceptional curves, so the basis is L, E[0], .. , E[n_blowups-1]
E = [N(e) for e in E]
Line = lambda exceptional_curves : N([i/3 for i in (-K + sum(exceptional_curves)) ])
L = Line(E)

Q = diagonal_matrix([1]+n_blowups*[-1])
dot = lambda a, b: a*Q*b
gram_matrix = lambda rays: matrix([[dot(a,b) for a in rays] for b in rays])

minus_one_curves = E + [L-ei-ej for ei,ej in itertools.combinations(E, 2)] + [2*L-sum(quintuple) for quintuple in itertools.combinations(E, 5)]
if(n_blowups>=7):
    for septuple in itertools.combinations(E, 7):
        for e in septuple:
            minus_one_curves.append(3*L-sum(septuple)-e)

curves_disjoint_with = lambda curves: [c for c in minus_one_curves if all(dot(c,c2)==0 for c2 in curves)]

double_intersections = [(a,b) for a,b in itertools.combinations(minus_one_curves, 2) if dot(a,b)>0]
triple_intersections = [(a,b,c) for a,b,c in itertools.combinations(minus_one_curves, 3) if dot(a,b)*dot(b,c)*dot(a,c)>0]

NE = Cone(minus_one_curves)
Ample = Cone([N(Q*ray) for ray in NE.dual().rays()])

# take one Fujita face for each isomorphism class (determined by CPW, may check explicitly later)
# and turn them into subcones in Mori cone by adding -K

def type_of_face(rays):
    self_intersections = [dot(r,r) for r in rays]
    if all (dot(a,b)==0 for a, b in itertools.combinations(rays,2)):        
        if all(s in [-1,0] for s in self_intersections):
            number_exceptional_curves = - sum(self_intersections)
            if number_exceptional_curves == len(rays):
                return f'B{len(rays)}'
            if number_exceptional_curves == len(rays)-1:
                return f'C{len(rays)-1}'
    if all(s==-1 for s in self_intersections):
        if all(sum(1 for t in rays if dot(t,r)==1) == 1 for r in rays):
            return f'C({len(rays)})'
    return 'unknown, gram:\n'+gram_matrix(rays).str()
        

NE_face_types_rays = {
        "B7": E[:7],
        "B6": E[:6],
        "B5": E[:5],
        "B5'": E[:4]+[L-E[4]-E[5]], # B(5) for P1xP1. Another type of independent set of 5 (-1)-curves, this one is maximal
        "B4": E[:4],
        "B3": E[:3],
        "B2": E[:2],
        "B1": E[:1],
        #'C6': E[:5]+[L-E[5]-e for e in E[:5]], # C(6). Schlafli graph is 4-ultrahomogeneous, and, after fixing two pairs, three others are defined uniquely
        "C5": E[:5]+[L-E[5]], # здесь и ниже - части C6, из подобных которым складывается весь С6 
        "C5'": E[:4]+[L-E[5]-E[4], L-E[5]],
        "C4": E[:4]+[L-E[5]],
        "C3": E[:3]+[L-E[5]],
        "C2": E[:2]+[L-E[5]],
        "C1": E[:1]+[L-E[5]],
        "C0": E[:0]+[L-E[5]],
    }
cone_of_type = {label:Cone(rays+[-K]).intersection(Ample) for label,rays in NE_face_types_rays.items()}

# returns True if rel.int.(cone1) is in rel.int.(cone2)
is_covered_by = lambda cone1, cone2: all(cone2.contains(ray) for ray in cone1.rays()) and cone2.interior_contains(sum(cone1.rays()))

In [3]:

def independent_sets(vectors, size = n_blowups):
    if size == 0:
        yield []
    for i, v in enumerate(vectors):
        orthogonals = [v2 for v2 in vectors[i+1:] if dot(v, v2)==0]
        for subset in independent_sets(orthogonals, size-1):
            yield subset + [v]


In [4]:
len(list(independent_sets(minus_one_curves, 7)))

576

In [5]:
for f in tqdm(NE.faces()[4]):
    t = type_of_face(f.rays())
    if "B" not in t and t!="C":
        print(gram_matrix(f.rays()))
        print(t)
        break
for i in range(8):
    print(collections.Counter(type_of_face(f.rays()) for f in NE.faces()[i]))

100%|██████████| 10080/10080 [00:08<00:00, 1206.21it/s]


Counter({'B0': 1})
Counter({'B1': 56})
Counter({'B2': 756})
Counter({'B3': 4032})
Counter({'B4': 10080})
Counter({'B5': 12096})
Counter({'B6': 6048})
Counter({'B7': 576, 'C(12)': 126})


Counter({'B0': 1})
Counter({'B1': 56})
Counter({'B2': 756})
Counter({'B3': 4032})
Counter({'B4': 10080})


KeyboardInterrupt: 